## 데이터 추출 여부 확인 부분

* 고려해야 하는 사항 중 Evt Log 마다 항목이 동일한 것이 아님 (Check 및 작업 필요) 

In [1]:
# Note Book 내 응용프로그램 Log 추출하는 작업 
import win32evtlog
import xml.etree.ElementTree as ET
from datetime import datetime
import re

In [2]:
# Window Evt Log 접속
file = win32evtlog.EvtQuery('C:\\Windows\\System32\\Winevt\\Logs\\Application.evtx',
                           win32evtlog.EvtQueryFilePath)

In [7]:
# Evt Log 내 Evt Log 1개씩 추출작업 > 해당 코드 실행결과 값이 없으면 Evt Log 끝 
log_extract = win32evtlog.EvtNext(file,1)

In [24]:
# Evt Log 추출 > XML Format 변환 전 TXT 형태로 추출 
xml_contents = win32evtlog.EvtRender(log_extract[0], win32evtlog.EvtRenderEventXml)

"<Event xmlns='http://schemas.microsoft.com/win/2004/08/events/event'><System><Provider Name='Microsoft-Windows-User Profiles Service' Guid='{89b1e9f0-5aff-44a6-9b44-0a07a7ce5845}'/><EventID>1531</EventID><Version>0</Version><Level>4</Level><Task>0</Task><Opcode>0</Opcode><Keywords>0x8000000000000000</Keywords><TimeCreated SystemTime='2023-05-05T07:32:12.8763468Z'/><EventRecordID>1</EventRecordID><Correlation/><Execution ProcessID='1600' ThreadID='1972'/><Channel>Application</Channel><Computer>WIN-2JD0CI7630G</Computer><Security UserID='S-1-5-18'/></System><EventData></EventData></Event>"

In [25]:
# Evt Log XML Format 적용
xml = ET.fromstring(xml_contents)
ns = '{http://schemas.microsoft.com/win/2004/08/events/event}' # 기본 작업 (xml Format 변환시 출력 문구 정보)
excution = xml.find(f'.//{ns}Execution')

<Element '{http://schemas.microsoft.com/win/2004/08/events/event}Execution' at 0x000001BE38F3DE90>

In [26]:
# Event ID Extract 
print("Event ID : {}".format(xml.find(f'.//{ns}EventID').text))
print("Level : {}".format(xml.find(f'.//{ns}Level').text))
print("Task : {}".format(xml.find(f'.//{ns}Task').text))
print("Time Created : {}".format(xml.find(f'.//{ns}TimeCreated').get('SystemTime')))
print("Channel : {}".format(xml.find(f'.//{ns}Channel').text))
print("Computer : {}".format(xml.find(f'.//{ns}Computer').text))
# Excution 항목
print("ProcessID : {}".format(excution.get('ProcessID')))
print("ThreadID : {}".format(excution.get('ThreadID')))

Event ID : 1531
Level : 4
Task : 0
Time Created : 2023-05-05T07:32:12.8763468Z
Channel : Application
Computer : WIN-2JD0CI7630G
ProcessID : 1600
ThreadID : 1972


## 스케줄 코드로 변환하는 작업 

In [59]:
import win32evtlog
import xml.etree.ElementTree as ET
from datetime import datetime
import re
import pandas as pd

# Window Evt Log 접속
file = win32evtlog.EvtQuery('C:\\Windows\\System32\\Winevt\\Logs\\Application.evtx',
                           win32evtlog.EvtQueryFilePath)
EventID = []; Level = []; TASK = []; CREATED_TIME = []; Channel = []; Computer = []; 
ProcessID = []; ThreadID = [];

Num = 0

while True :
    Num += 1
    
    # Evt Log 내 Evt Log 1개씩 추출작업 > 해당 코드 실행결과 값이 없으면 Evt Log 끝 
    log_extract = win32evtlog.EvtNext(file,1)
    
    if len(log_extract) == 0:
        break 
        
    # Evt Log 추출 > XML Format 변환 전 TXT 형태로 추출 
    xml_contents = win32evtlog.EvtRender(log_extract[0], win32evtlog.EvtRenderEventXml)
    
    # Evt Log XML Format 적용
    xml = ET.fromstring(xml_contents)
    ns = '{http://schemas.microsoft.com/win/2004/08/events/event}' # 기본 작업 (xml Format 변환시 출력 문구 정보)
    excution = xml.find(f'.//{ns}Execution')
    
    ext_eventid = xml.find(f'.//{ns}EventID').text
    ext_level = xml.find(f'.//{ns}Level').text
    ext_task = xml.find(f'.//{ns}Task').text
    ext_created_time = re.sub('T',' ',xml.find(f'.//{ns}TimeCreated').get('SystemTime')[:19])
    ext_channel = xml.find(f'.//{ns}Channel').text
    ext_computer = xml.find(f'.//{ns}Computer').text
    ext_ProcessID = excution.get('ProcessID')
    ext_ThreadID = excution.get('ThreadID')
    
    EventID.append(ext_eventid)
    Level.append(ext_level)
    TASK.append(ext_task)
    CREATED_TIME.append(ext_created_time)
    Channel.append(ext_channel)
    Computer.append(ext_computer)
    ProcessID.append(ext_ProcessID)
    ThreadID.append(ext_ThreadID)

In [61]:
data = pd.DataFrame({'CREATED_TIME':CREATED_TIME, 'EVENT_ID':EventID, 'LEVEL':Level, 'TASK':TASK, 'CHANNEL':Channel,
                        'PROCESSID':ProcessID,'ThreadID':ThreadID})

In [62]:
dir_ = 'D:\Window_Event_Extract\\data\\WindowLOG{}.csv'.format(re.sub('-','',str(datetime.now())[:10]))
data.to_csv(dir_,index=False)